<h1>A few examples of looking at location of interest and selected tracks  </h1>

load libraries

In [1]:
from icepyx import icesat2data as ipd
import numpy as np
import os
import shutil
import h5py
import cartopy.crs as ccrs
import sys
import matplotlib.pyplot as plt
import pyproj
%matplotlib widget

load file that has already been downloaded

In [2]:

FILE_NAME = '/home/jovyan/shared/data-showard/processed_ATL06_20200330121520_00600712_003_01.h5'
f = h5py.File(FILE_NAME, mode='r') 

In [ ]:
# Inspect file from the command line
!h5ls -r /home/jovyan/shared/data-showard/processed_ATL06_20200330121520_00600712_003_01.h5

/                        Group
/METADATA                Group


Pull out some variables to look at

In [ ]:


count = f['gt1l/residual_histogram/count'] # has units of n_histograms, n_bins
lat_mean = f['gt1l/residual_histogram/lat_mean']
lon_mean = f['gt1l/residual_histogram/lon_mean']
h_li = f['gt1l/land_ice_segments/h_li']
h_lat = f['gt1l/land_ice_segments/latitude']
h_lon = f['gt1l/land_ice_segments/longitude']


Plot histogram count data usign method from PlotPhotoHistrograms.ipynb

In [ ]:
data = np.transpose(np.array(count))

In [ ]:

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
h = ax.imshow(data,vmin=0,vmax=30,cmap='Reds')
plt.colorbar(h)
plt.show()

<h2>Plot Search area on Moa image </h2>

** still needs track on this image

In [ ]:
data_root='/srv/tutorial-data/land_ice_applications/'

In [ ]:
! cd ..; [ -d pointCollection ] || git clone https://www.github.com/smithB/pointCollection.git
sys.path.append(os.path.join(os.getcwd(), '..'))
import pointCollection as pc

In [ ]:
spatial_extent = np.array([25, -72.56, 40, -67])
lat=spatial_extent[[1, 3, 3, 1, 1]]
lon=spatial_extent[[2, 2, 0, 0, 2]]
print(lat)
print(lon)
# project the coordinates to Antarctic polar stereographic
xy=np.array(pyproj.Proj(3031)(lon, lat))
# get the bounds of the projected coordinates 
XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
MOA=pc.grid.data().from_geotif(os.path.join(data_root, 'MOA','moa_2009_1km.tif'), bounds=[XR, YR])

# show the mosaic:
plt.figure()
MOA.show(cmap='gray', clim=[14000, 17000])
plt.plot(xy[0,:], xy[1,:])
plt.title(' Area used for search')

<h2> A different way of plotting track on map</a>

In [ ]:


## we will want to set colorbar parameters based on the chosen variable
vmin=0
vmax=6
ticks=np.arange(vmin,vmax+1,1)

plt.figure(figsize=(8,8), dpi= 90)
ax = plt.axes(projection=ccrs.SouthPolarStereo(central_longitude=0)) # choose polar sterographic for projection
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.set_extent([25 , 40, -73, -67], ccrs.PlateCarree())
plt.plot(lon_mean,lat_mean,transform=ccrs.PlateCarree())
plt.show()


Plot comparing mean_lon and mean_lon from histrograms with beam lat and lon

In [ ]:
plt.figure()
plt.plot(h_lon,h_lat,'ob' )
plt.plot(lon_mean, lat_mean,'.r')